In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import json
import folium

In [2]:
tree_dat = pd.read_csv( 'trees2015.csv' )#, dtype=str )
#tree_dat.info()

In [3]:
cols_4Qs = [ 'spc_common', 'boroname', 'health', 'steward', 'zipcode', 'zip_city', 'borocode', 'latitude', 'longitude' ]

tree_dat_4Qs = tree_dat[ cols_4Qs ]
print( tree_dat_4Qs.head() )
print( tree_dat_4Qs.info() )

        spc_common  boroname health steward  zipcode      zip_city  borocode  \
0        red maple    Queens   Fair    None    11375  Forest Hills         4   
1          pin oak    Queens   Fair    None    11357    Whitestone         4   
2      honeylocust  Brooklyn   Good    1or2    11211      Brooklyn         3   
3      honeylocust  Brooklyn   Good    None    11211      Brooklyn         3   
4  American linden  Brooklyn   Good    None    11215      Brooklyn         3   

    latitude  longitude  
0  40.723092 -73.844215  
1  40.794111 -73.818679  
2  40.717581 -73.936608  
3  40.713537 -73.934456  
4  40.666778 -73.975979  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   spc_common  652169 non-null  object 
 1   boroname    683788 non-null  object 
 2   health      652172 non-null  object 
 3   steward     652173 non-null  object 
 

In [4]:
print( set( tree_dat_4Qs['zipcode'] ) )
print( set( tree_dat_4Qs['spc_common'] ) )

{10280, 10281, 10282, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10312, 10314, 83, 11354, 11355, 11356, 11357, 11358, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11377, 11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429, 11430, 11432, 11433, 11434, 11435, 11436, 11451, 10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465, 10466, 10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475, 10020, 11001, 11004, 11005, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10017, 10018, 10019, 11040, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10048, 10065, 10069, 11096, 10075, 11101, 11102, 11103, 11104, 11105, 11106, 1

In [57]:
decendingTrees = tree_dat_4Qs.groupby('spc_common').size().reset_index(name='counts').sort_values( 'counts' )

decendingTrees.iloc[30:60]

,spc_common,counts
66,cucumber magnolia,169
106,sassafras,171
13,Chinese chestnut,173
99,pond cypress,181
7,Amur cork tree,183
130,white pine,202
117,spruce,202
35,Oklahoma redbud,219
94,paperbark maple,220
126,two-winged silverbell,221


In [58]:
NY_coords = [ 40.7128, -74.0060 ]
#CartoDB” (positron and dark_matter) “OpenStreetMap”
#“Stamen” (Terrain, Toner, and Watercolor)
ny_map = folium.Map( location = NY_coords, tiles = 'Stamen Watercolor', zoom_start = 10 )
ny_map

In [34]:
from folium.plugins import MarkerCluster
def map_certain_trees( df, tree_species ):
    df_species = df[ df[ 'spc_common' ] == tree_species ]
    marker_cluster = MarkerCluster().add_to( ny_map )
    for idx, row in df_species.iterrows():
        marker = folium.Marker( location = [row['latitude'], row['longitude']],
                              icon=folium.Icon(color='green', 
                                               icon='tree', prefix='fa')).add_to( marker_cluster )

In [59]:
map_certain_trees( tree_dat_4Qs, 'crepe myrtle' )

In [60]:
ny_map

In [ ]:
idx = 0
[df_species['latitude'].iloc[idx], df_species['longitude'].iloc[idx]]